In [1]:
from os import listdir
from os.path import isfile, join
from PIL import Image

directory = '/home/mick/Downloads/test2017-20201117T095244Z-001/test2017/'
optDir = '/home/mick/Desktop/gpufilesystem/dataset/'

files = [f for f in listdir(directory) if isfile(join(directory, f))]
count = 0
'''
for fileName in files:
    filePath = directory + fileName
    im = Image.open(filePath)
    im = im.convert(mode='RGB', colors=256)
    size = 128, 128
    im = im.resize(size)
    im.save(optDir + str(count), format='JPEG')
    count += 1
'''

"\nfor fileName in files:\n    filePath = directory + fileName\n    im = Image.open(filePath)\n    im = im.convert(mode='RGB', colors=256)\n    size = 128, 128\n    im = im.resize(size)\n    im.save(optDir + str(count), format='JPEG')\n    count += 1\n"

In [2]:
from PIL import Image, ImageDraw,ImageChops
import numpy as np
from os import listdir
from os.path import isfile, join
import time
from datetime import datetime

def crop(input, height, width):
    opt = []
    k = 0
    im = input
    imgwidth, imgheight = im.size
    for i in range(0,imgheight,height):
        for j in range(0,imgwidth,width):
            box = (j, i, j+width, i+height)
            draw = ImageDraw.Draw(im)
            draw.rectangle(box)
            a = im.crop(box)
            a.save(optDir + str(k) + '.jpg')
            k+=1
            opt.append(a)
    im.show()
    return opt

def get_concat_v_multi_resize(im_list, resample=Image.BICUBIC):
    min_width = min(im.width for im in im_list)
    im_list_resize = [im.resize((min_width, int(im.height * min_width / im.width)),resample=resample)
                      for im in im_list]
    total_height = sum(im.height for im in im_list_resize)
    dst = Image.new('RGB', (min_width, total_height))
    pos_y = 0
    for im in im_list_resize:
        dst.paste(im, (0, pos_y))
        pos_y += im.height
    return dst

def get_concat_h_multi_resize(im_list, resample=Image.BICUBIC):
    min_height = min(im.height for im in im_list)
    im_list_resize = [im.resize((int(im.width * min_height / im.height), min_height),resample=resample)
                      for im in im_list]
    total_width = sum(im.width for im in im_list_resize)
    dst = Image.new('RGB', (total_width, min_height))
    pos_x = 0
    for im in im_list_resize:
        dst.paste(im, (pos_x, 0))
        pos_x += im.width
    return dst

def get_concat_tile_resize(im_list_2d, resample=Image.BICUBIC):
    im_list_v = [get_concat_h_multi_resize(im_list_h, resample=resample) for im_list_h in im_list_2d]
    return get_concat_v_multi_resize(im_list_v, resample=resample)

start = time.time()
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time :", current_time)

dataDir = '/home/mick/Desktop/gpufilesystem/dataset/'
optDir = '/home/mick/Desktop/gpufilesystem/examples/CPU/opt/'
files = [f for f in listdir(optDir) if isfile(join(optDir, f))]
for file in files:
    import os
    os.remove(optDir + file)

files = [f for f in listdir(dataDir) if isfile(join(dataDir, f))]
im = Image.open(dataDir + '4')
im = im.resize((1024,1024))

#has to be power of 2
dim = (32, 32, 3)

matrix_width = int(im.size[0] / dim[0])
matrix_height = int(im.size[1] / dim[1])

image_segments = crop(im, matrix_width, matrix_height)

vertical = []
for x in range(dim[0]):
    horizontal = []
    for y in range(dim[1]):
        target_index = x * dim[0] + y
        #print(x,y,target_index, end = ', ')
        print('.', end = '')
        diff_list = []
        target = Image.open(optDir + str(target_index) + '.jpg')
        #target.show()
        for file_index in range(len(files)):
            candidate = Image.open(dataDir + files[file_index]).resize(target.size)
            diff = ImageChops.difference(target, candidate)
            array = np.array(diff)
            diff_list.append(array.mean())
        path = dataDir + files[diff_list.index(min(diff_list))]
        result = Image.open(path)
        horizontal.append(result)
    vertical.append(horizontal)
opt_img = get_concat_tile_resize(vertical)
opt_img = opt_img.resize((1024,1024))
opt_img.save('opt.jpg')
opt_img.show()

done = time.time()
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Finish Time :", current_time)
elapsed = done - start
print('Processing used (s): ', elapsed)

Start Time : 20:09:57
..................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................